In [1]:
import import_ipynb
import math
import numpy as np
import pandas as pd
from collections import Counter
import datetime
import time
import openpyxl
import os
from tqdm import tqdm
from scipy import stats
import missingno as msno
from snn import SyntheticNearestNeighbors
from sklearn.pipeline import Pipeline

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow_addons.metrics import RSquare
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import DenseFeatures
from tensorflow.keras import Input
from tensorflow.feature_column import make_parse_example_spec
from tensorflow.feature_column import numeric_column
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.constraints import MaxNorm
from scikeras.wrappers import KerasRegressor

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from mpl_toolkits import mplot3d
import seaborn as sns
sns.set_style("whitegrid")
sns.color_palette("pastel")

from scipy.optimize import minimize
from scipy.optimize import LinearConstraint


2022-11-03 20:27:26.215067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-03 20:27:26.215100: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def timer(start_time=None, string=None):
    '''
    Function to compute the time
    start_time : starting time generated calling this function without arguments the first time
    string: visualization purposes (task description)
    '''
    if not start_time:
        start_time=datetime.datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.datetime.now()-start_time).total_seconds(),3600)
        tmin, tsec = divmod(temp_sec,60)
        pr = "for " + str(string) + " " if string else ""
        print("Execution time", pr, "is ", thour," h :", tmin,' m :', round(tsec,2), " s")

### Generation simulated data

In [3]:
def dataset_LCM(P, J, K, MoyLCM, Mass0, N_tableaux, Weight_fixed=False, snn=False):
    '''
    Function for the creation of the complete matrix (still to be preprocessed)
    P : size of population
    J: number de devices
    K : Number of user types
    MoyLCM : Mean of the GMM
    N_tableaux : number of sub-campaigns (n. of tableaux to generate)
    Weight_fixed : if True, the Dirac weights in 0 are all equal to Mass0
                    if False, the weights are chosen in [0, 1] stochastically
    '''
    # Init
    users=[i for i in range(1,P+1)]
    Tableau={}
    alpha_proportion={}
    True_Z={}
    alpha={}
    theta=np.zeros((K,J))
    
    # Weights of Dirac to induce sparsity
    if Weight_fixed:
        poids_0=Mass0*np.ones((K,J))  # used as threshold to fill with 0 (compared to np.random.rand for the decision)
    else:
        poids_0=Mass0*np.random.rand(K,J)
    print(f'{poids_0} are the thresholds for sparsity \n')
    
    # Loop over types to create parameters of K*J Poissons
    for k in range(K):  
        A = np.random.rand(J,J)  # since dimension of Mixture Gaussian is J
        B = np.dot(A,A.transpose())   # variance eof Mixture Gaussian (random and definitive positive) diff x type
        alpha[k]=np.random.multivariate_normal(np.array([MoyLCM for m in range(J)]),B)  # centre is (moy)*J_dim
        for j in range(len(alpha[k])):
            theta[k][j]=np.exp(alpha[k][j])  # parameters of poisson 
    print(f'{theta} are the parameters for the KxJ Poissons \n')
        
    # Creation of subcampaigns (full matrices)
    for n_t in range(N_tableaux):
        d={}
        # random assignment of types in [0, K] for each user of each subcampaign
        True_Z[n_t]=np.random.randint(0,K,size=P)
        alpha_proportion[n_t]=[0 for i in range(K)]
        
        Z=True_Z[n_t]
        # storing of proportion of types
        for i in Counter(Z):   # (not) sorted
            alpha_proportion[n_t][i]=Counter(Z)[i]/len(Z)    
        alpha_proportion[n_t]=np.array(alpha_proportion[n_t])
        
        users=[n_t*P+i for i in range(1,P+1)]   # list with unique id for P users of Tableau that is being generated
        d['users']=users
        
        for j in range(J):
            d['J'+str(j+1)]=[]   # d = {'users': [203, 204, ..], 'J1': [], 'J2': [], ..., 'up to J': []}
            
        for i in range(P):
            k0=True_Z[n_t][i]
            for j in range(J):
                if np.random.rand()<=poids_0[k0][j]:  # Dirac weights tell probability of sparsity (filled with 0)
                    d['J'+str(j+1)]+=[np.nan] if snn==True else [0]                        
                else:
                    d['J'+str(j+1)]+=[np.random.poisson(theta[k0][j])]  # else used appropriate Poisson parameter (x user type & x device) to fill
                    
        # storing
        df_sans_independance=pd.DataFrame.from_dict(d)
        Tableau[n_t]=df_sans_independance[['users']+['J' +str(j+1) for j in range(J)]]
      
    return Tableau, True_Z, theta, alpha_proportion, poids_0


In [4]:
def reach_fun_RI(Nuj): #Fonction pour calculer le reach sur un tableau numpy
    '''
    Function to compute the reach function (in percentage over P) given the matrix of a subcampaign
    '''
    P=len(Nuj)
    reach_true=0
    for u in range(P):
        if np.sum(Nuj.values[u][1:])>0:
            reach_true+=1.0/P
    return reach_true

def preprocess_dataset(Tableau, prop_panel=1):
    '''
    (Timed) function to compute the reach function and the normalized vector of sum of cookies
    '''
    P = Tableau[0].shape[0]
    J = Tableau[0].shape[1]-1
    
    X_panel, Y_panel = [], []
    
    assert prop_panel<=1,"The proportion of panelists must be lower (or equal) than 1"
    result = {}
    
    # start timer
    start = timer()
    
    # computations reach and vector of counts for each sub-campaign
    for n in range(len(Tableau)):
        T_aux=Tableau[n].to_numpy()
        reach_panelist=reach_fun_RI(Tableau[n][Tableau[n]['users']<=(n*P)+P*prop_panel])

        c_panelist=list(Tableau[n][Tableau[n]['users']<=(n*P)+P*prop_panel].sum()/(P*prop_panel) )[1:]  # /P for normalization of vector count
        c_non_panelist=[(Tableau[n].sum()/(P*(1-prop_panel)))[i+1]-c_panelist[i] for i in range(J)] if prop_panel!=1 else [0 for i in range(J)]
        
        # result stores everything, separately we build regressors (normalized vector of cookies count) and dependent (calculation of reach function)
        result[n]=(T_aux, c_panelist, reach_panelist, c_non_panelist)
        X_panel.append(np.array(c_panelist))  
        Y_panel.append(np.array(reach_panelist))
        
    # regressors
    X = np.reshape(X_panel,(len(Tableau),J))  # transform in arrays
    
    # target
    Y = np.reshape(Y_panel, (len(Tableau), 1))
    
    # visualization of timer with description of task
    timer(start, "preprocessing the dataset")
    
    return result, X, Y


In [ ]:
def create_features(X_train, X_test, Y_train, Y_test):
    ''' Function used to create engineered features as outputs of a deep neural network'''
    
    clear_session()
    
    model=Sequential()  #name
    model.add(BatchNormalization(input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(30, name='first_hidden', kernel_constraint=MaxNorm(3.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(100, name='second_hidden', kernel_constraint=MaxNorm(3.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(30, name='third_hidden', kernel_constraint=MaxNorm(3.0)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='output'))
    model.compile(optimizer=RMSprop(momentum=0.9), loss=MeanSquaredError(), metrics=[RSquare(), RootMeanSquaredError()])  # loss_weights = ..., 
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, verbose=0, batch_size=32)
    model.summary()
    
    wanted_layer = 'second_hidden'
    feature_model = Model(inputs=model.input, outputs=model.get_layer(wanted_layer).output)
    
    addit_feature_train = feature_model.predict(X_train)  # dropout not used when predictions, it knows in test mode automatically
    addit_feature_test = feature_model.predict(X_test)
    X_train_eng = np.concatenate((X_train, addit_feature_train), axis=1)
    X_test_eng = np.concatenate((X_test, addit_feature_test), axis=1)
    print(f"Engineered X_train with number of features = {X_train_eng.shape[1]}")
    print(f"Engineered X_test with number of features = {X_test_eng.shape[1]}")
    return X_train_eng, X_test_eng


In [ ]:
def create_1hidden_model(batch_norm=False, dropout_rate_init=0, dropout_rate_hidd=0, init_mode='uniform', activ_hidd='relu',
                         activ_out='linear', constraint=5.0, neurons=10, loss_=MeanSquaredError()):
    '''
    feature_outputs is output of above function (dense Tensors)
    '''
    clear_session()
    
    model=Sequential()  #name
    if batch_norm==True:
        model.add(BatchNormalization(input_shape=(input_dim,)))
        model.add(Dropout(dropout_rate_init))
    else: 
        model.add(Dropout(dropout_rate_init, input_shape=(input_dim,)))
    model.add(Dense(neurons, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_hidd))
    model.add(Dense(1, activation=activ_out, kernel_initializer=init_mode, name='output'))
    model.compile(optimizer=RMSprop(momentum=0.9), loss=loss_, metrics=[RSquare(), RootMeanSquaredError()])  # loss_weights = ..., 
    return model

In [ ]:
def create_2hidden_model(batch_norm=False, dropout_rate_init=0, dropout_rate_hidd=0, init_mode='uniform', activ_hidd='relu',
                         activ_out='linear', constraint=5.0, neurons_1=10, neurons_2=5, loss_=MeanSquaredError()):
    '''
    feature_outputs is output of above function (dense Tensors)
    '''
    clear_session()
    
    model=Sequential()  #name
    if batch_norm==True:
        model.add(BatchNormalization(input_shape=(input_dim,)))
        model.add(Dropout(dropout_rate_init))
    else: 
        model.add(Dropout(dropout_rate_init, input_shape=(input_dim,)))
    model.add(Dense(neurons_1, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer_1', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_hidd))
    model.add(Dense(neurons_2, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer_2', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_hidd))
    model.add(Dense(1, activation=activ_out, kernel_initializer=init_mode, name='output'))
    model.compile(loss=loss_, metrics=[RSquare(), RootMeanSquaredError()])  # loss_weights = ..., 
    return model

In [ ]:
def create_3hidden_model(batch_norm=False, dropout_rate_init=0, dropout_rate_hidd=0, init_mode='uniform', activ_hidd='relu',
                         activ_out='linear', constraint=5.0, neurons_1=10, neurons_2=5, neurons_3=2, loss_=MeanSquaredError()):
    '''
    feature_outputs is output of above function (dense Tensors)
    '''
    clear_session()
    
    model=Sequential()  #name
    if batch_norm==True:
        model.add(BatchNormalization(input_shape=(input_dim,)))
        model.add(Dropout(dropout_rate_init))
    else: 
        model.add(Dropout(dropout_rate_init, input_shape=(input_dim,)))
    model.add(Dense(neurons_1, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer_1', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_hidd))
    model.add(Dense(neurons_2, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer_2', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    model.add(Dropout(dropout_rate_hidd))
    model.add(Dense(neurons_3, activation=activ_hidd, kernel_initializer=init_mode, name='hidden_layer_3', kernel_constraint=MaxNorm(constraint)))
    if batch_norm==True:
        model.add(BatchNormalization())
    # without final Dropout
    model.add(Dense(1, activation=activ_out, kernel_initializer=init_mode, name='output'))
    model.compile(loss=loss_, metrics=[RSquare(), RootMeanSquaredError()])  # loss_weights = ..., 
    return model

In [ ]:
def plot_nets(history):
    rmse_lst = history.history['root_mean_squared_error']
    val_rmse_lst = history.history['val_root_mean_squared_error']
    
    r2_lst = history.history['r_square']
    val_r2_lst = history.history['val_r_square']
                  
    fig, ax = plt.subplots(2, 1, figsize=(12, 10))
    x_axis=range(1, len(rmse_lst)+1)  # epochs
    ax[0].plot(x_axis, rmse_lst, 'r', label="Train dataset")
    ax[0].plot(x_axis, val_rmse_lst, 'b', label="Evaluation dataset")
    ax[0].set_title("Evolution of RMSE in Training and Test dataset")
    ax[1].plot(x_axis, r2_lst, 'r', label="Train dataset")
    ax[1].plot(x_axis, val_r2_lst, 'b', label="Evaluation dataset")
    ax[1].set_title("Evolution of R2 in Training and Test dataset")
    ax[0].legend()
    ax[1].legend()
    plt.show()

In [ ]:
def exp_decay(epoch):
    lrate= lr * np.exp(-decay*epoch)
    return lrate 

### Generation real Data

In [5]:
def generate_real_data(resp_level, d_group=None):
    '''
    resp_level:  Mediametrie data as provided (csv file)
    d_group: if None computed for all dem_group, otherwise list with demographic ID to generate data for
    
    output: dict with keys = demographic group ID and 
                 and values = dict with keys = n. of campaigns and 
                                        values = dict as 
                            {'full': entire N_uj matrix, 'reach': int, n. of unique user reached (normalized by P), 
                            'sum': result_campaign.sum(axis=0)/result_campaign.shape[0] }   
    '''
    
    # start timer
    start = timer()
    
    dem_group = sorted(resp_level["Demographic Group"].unique().tolist()) if d_group==None else sorted(d_group)
    fig, ax = plt.subplots(math.ceil(len(dem_group)/2), 2, figsize=(14, 7*math.ceil(len(dem_group)/2)))
    
    campaigns = sorted(resp_level["Campaign ID"].unique().tolist())
    cookies_type = resp_level["Station"].unique().tolist()
    cookies_dict = {cookies_type[i]: list(range(len(cookies_type)))[i] for i in range(len(cookies_type))}  # dict for Station (cookies types)

    final_store = {}

    for group in dem_group:
        print(f"\nDemographic group: {group}")
        number_rows_tot, reach_print = [], []
        campaign_store = {}
        to_plot = {list(range(1, len(cookies_type)+1))[i]: [0, 0.0] for i in range(len(cookies_type))}
        # n. of cookies used by campaign (at least one observation): [count campaigns, sum of reach]
        
        dem_slice = resp_level[resp_level["Demographic Group"]==group]
        for campaign in campaigns:
            camp_slice = dem_slice[dem_slice["Campaign ID"]==campaign]
            assert camp_slice.shape[0]>0, "better check"  # all campaign should be in all demographic group

            serial_lst = sorted(list(camp_slice["Serial"].unique()))
            weight_lst = camp_slice.sort_values(by="Serial", axis=0, inplace=False) \
                            .groupby("Serial")["Weight"].unique().reset_index()["Weight"].astype("int").values
            weight_cum = np.cumsum(weight_lst)
            weight_dict = {serial_lst[i]: weight_lst[i] for i in range(len(serial_lst))} # to find subset of lines for specific Serial
            weight_dict_cum = {serial_lst[i]: weight_cum[i] for i in range(len(serial_lst))} # to find subset of lines for specific Serial

            result_campaign = np.zeros((weight_cum[-1], len(cookies_type)))  # total matrix 
            # rows = (weights are only those inside campaign, depending on Serial) & columns = (made of all cookied types, even not used)
            number_rows_tot.append(weight_cum[-1])
            
            cookies_printed = len(camp_slice["Station"].unique().tolist())
            to_plot[cookies_printed][0] += 1

            for u, serial in enumerate(serial_lst):
                serial_slice = camp_slice[camp_slice["Serial"]==serial]

                top_r = 0 if u==0 else weight_dict_cum[serial_lst[u-1]]
                bottom_r = weight_dict_cum[serial]
                slice_rows = np.arange(top_r, bottom_r, dtype=int)

                cookies_used = serial_slice["Station"].unique().tolist()
                for stat in cookies_used:
                    station_slice = serial_slice[serial_slice["Station"]==stat]
                    
                    prop_viewed = station_slice["Proportion of Ad Break viewed"].values
                    tot_avg, tot_left = 0, 0
                    for viewed in prop_viewed:  # Ads Break are not even filtered by
                        num = viewed*weight_dict[serial]
                        avg, addit = divmod(num, weight_dict[serial])                    
                        tot_avg += avg
                        tot_left += addit

                    final_n, final_left = divmod(np.rint(tot_left), weight_dict[serial])
                    tot_avg += final_n

                    random_rows = np.random.choice(slice_rows, size=np.rint(final_left).astype(int), replace=False)
                    result_campaign[slice_rows, cookies_dict[stat]] += tot_avg
                    result_campaign[random_rows, cookies_dict[stat]] += 1 
            
            reach = 0.0
            for row in range(result_campaign.shape[0]):
                if result_campaign[row].sum() > 0:
                    reach += 1
            reach /= result_campaign.shape[0]
            reach_print.append(reach)

            campaign_store[campaign] = {'full': np.copy(result_campaign), 
                                        'reach': reach,
                                        'sum': result_campaign.sum(axis=0)/result_campaign.shape[0]
                                       }            
                
            to_plot[cookies_printed][1] += reach
        
        avg_reach_station = []
        for n_station_seen, dat in to_plot.items():
            avg_reach_station.append(dat[1]/float(dat[0]))   # mean of reach for each unique station seen (fixing demographic group))
                                     
        len_station_seen = np.array(list(to_plot.keys()))
        avg_reach_station = np.array(avg_reach_station)
        
        axs = [x for row in ax for x in row] # flatten for plot                     
        axs[dem_group.index(group)].plot(len_station_seen, avg_reach_station, linewidth=2)
        axs[dem_group.index(group)].set_xlabel('N. distinct TV-channels seen within a campaign', fontsize=15)
        axs[dem_group.index(group)].set_ylabel(r'Average estimated reach across campaign', fontsize=15)
        axs[dem_group.index(group)].set_title(f"Dem group {group}", fontsize=20)      
        
        final_store[group] = campaign_store
        
        number_rows_avg = np.array(number_rows_tot).mean()
        reach_print_avg = np.array(reach_print).mean()
        print(f"Average number of users observed (P): {int(number_rows_avg)}")
        print(f"Average reach/P observed (Reach): {reach_print_avg: .5f}\n")
    
    plt.tight_layout()
    plt.show()
     
    timer(start, "preprocessing Mediametrie data")
    return final_store, fig           


In [6]:
def split_real_data(final_store, test=False):
    '''
    function used for ML model data generation from Mediametrie data
    final_store: complex dict, result of generate_real_data
    test: bool to indicate whether we are processing October (train set) or January (test), 
          as they have different number of campaigns (there is an assert to be sure)
    
    returns: dict with keys=demographic ID and values=list of couples of np.arrays X, Y, 
             where X is sum of normalized sum of cookies count and Y is the estimated reach/P
             (for each of the demographic groups in the final_store provided)
    
    '''
    final = {}
    for dem_group, df_camp in final_store.items():
        X_pan, Y_pan = [], []
        for n_camp, data in df_camp.items():
            X_pan.append(data['sum'])
            Y_pan.append(float(data['reach']))
        n_camp = len(df_camp.keys())
        if test:
            assert n_camp == 418, 'better check'  # each demographic group should exhibit all 544 campaigns
        else: 
            assert n_camp == 544, 'better check'  # each demographic group should exhibit all 544 campaigns
        
        # regressors
        X = np.reshape(X_pan,(n_camp, len(X_pan[-1])))  # transform in arrays
        Y = np.reshape(Y_pan,(n_camp, 1)) 
        
        final[dem_group] = [X, Y]
    return final

### Synthetic Nearest Neighbors

In [7]:
def discretize_columns(Tab):
    '''
    function used to insert 0-1 on top and normalize columns by their sum (each user)
    Tab: np.array already transposed
    '''
    top_pad = []
    P = Tab.shape[1]
    w_us = Tab.astype("float")   # without users
    
    for col in range(P):
        col_sum=np.nansum(w_us[:, col]).astype("float")
        if col_sum==0:
            top_pad.append(0)
        else:
            w_us[:, col] = np.divide(w_us[:, col], col_sum)  # each column (user, with cookies=row since .T) is normalized with the sum
            top_pad.append(1)
    final = np.vstack((np.array(top_pad), w_us))
    return final

def sample_panelist(True_Z, alpha_proportion, P_panelist):
    '''
    Function used looping over all campaigns to obtain a number of panelist from a bigger dataset preserving
    the a priori proportion of user types
    

    True_z: is np array of size P (it is one value of a bigger dict output of dataset_LCM, with keys = Campaign id), 
              made of random int between 0 and (true) K
    alpha_proportion: is np array of size K, again as above it is a value of a bigger dict passed to the function
    P panelist: are the a priori wanted number of panelists (there are rounding errors 
                as proportion*Panelist is approximated to an integer)
    '''
    
    real_K = len(alpha_proportion)
    
    random_cols=np.array([]).astype(int)
    
    prosp_number = [np.rint(alpha_proportion[i]*P_panelist) for i in range(real_K)]
    for i in range(real_K):
        idx_type = np.where(True_Z==i)[0]
        selected = np.random.choice(idx_type, size=int(prosp_number[i]), replace=False) 
        random_cols = np.concatenate((random_cols, selected.astype(int)), axis=None)
    return random_cols

def snn_dataset_sim(Tableau, True_Z, alpha_proportion, normalize_sum_reach=False, P_panelist=100):
    '''
    (Timed) function that takes as input the simulated dataset (dict with keys=campaign ID)
    used to add 1st row filled with {0, 1} and last column with sum of collect cookies (for each j in J since transposed)
    
    the first value in the last column (position 0) is therefore the reach computed as int, which is stored and 
    substituted with a NaN value to be predicted by the SNN algorithm
    
    normalize_sum_reach: bool, whether to normalized last column (c mapped to t, and reach to reach/P)
    '''
    
    P = Tableau[0].shape[0]
    P_panelist = P if P<P_panelist else P_panelist
    J = Tableau[0].shape[1]-1  # there is 'user' column
    
    # start timer
    start = timer()
    result = {}
    
    # computations reach and vector of counts for each sub-campaign
    for n in range(len(Tableau)):
        T_aux = Tableau[n].to_numpy().T  # transposed
        T_aux = discretize_columns(T_aux[1:, :])  # discretization (first row)
        
        vector_sum=np.nansum(T_aux, axis=1).reshape((-1, 1))
        if normalize_sum_reach:
            vector_sum = np.divide(vector_sum, P)   # normalized t_i vector and reach 
        
        reach_comp = float(vector_sum[0, 0])
        vector_sum[0, 0] = np.nan   
        
        #slice_columns = np.arange(P, dtype=int)   # not taken care of rappresentation of group K
        #random_cols = np.random.choice(slice_columns, size=P_panelist, replace=False)
        random_cols = sample_panelist(True_Z[n], alpha_proportion[n], P_panelist)
        T_panelist = T_aux[:, random_cols]
           
        final = np.hstack((T_panelist, vector_sum))
        
        # result stores everything, separately we build regressors (normalized vector of cookies count) and dependent (calculation of reach function)
        result[n]=(final, reach_comp)
    
    # visualization of timer with description of task
    timer(start, "preprocessing the dataset for SNN")    
    return result

    

In [8]:
def snn_dataset_real(Tableau, norm_sum_reach=False, P_panelist=100):
    '''
    function that calls the previous function to handle Mediametrie data = dict with keys: Demographic group ID
    (there is the extra layer of demographic group, in addition to the campaign one)
    
    norm_sum_reach: calls parameter of above function
    P = n. of rows to select (number of panelist)
    '''
    final = {}    
    for dem_group, df_camp in Tableau.items():
        camp_store = {}
        reach_dict = {i: df_camp[i]['reach'] for i in df_camp.keys()}
        vector_sum_dict = {i: np.concatenate((np.array(np.nan), df_camp[i]['sum']), axis=None) for i in df_camp.keys()}
        users_dict  = {i: df_camp[i]['full'].shape[0] for i in df_camp.keys()}
        
        for n_camp, camp_mat in df_camp.items():
            # !!! here remains random, need to implement True_Z for real data
            slice_rows = np.arange(users_dict[n_camp], dtype=int)   # not taken care of rappresentation of group K
            random_rows = np.random.choice(slice_rows, size=P_panelist, replace=False)
            raw_mat = camp_mat[random_rows, :].T   # transposed
            raw_two = discretize_columns(Tab)
            
            final = np.hstack((raw_two, vector_sum_dict[n_camp]))
            camp_store[n_camp] = (final, reach_dict[n_camp])
            
        final[dem_group] = camp_store   # engmax is dict with keys=campaign ID and values (full matrix, normalized reach) tuple
    return final    

In [9]:
def snn(Tab, model, P_initial, n_comp=None):
    '''
    function implementing Synthethic Nearest Neighbour algorithm for our specific setup
    Tab: dict result of preprocessing (with keys=campaigns)
    model: sklearn estimator, with fit and predict methods
    P_initial is number of panelist + number of non panelist (Tab is made of panelist + vector count for both)
    n_components: if None all components are tried and the evolution of mean squared error is plotted
                  if fixed to a number (say n), considered only first n components
    '''
    r2, rmse = [], []
    
    P = Tab[0][0].shape[1]-1
    scaling_factor = P_initial/P
    
    n_campaigns = len(Tab)
    max_comp = Tab[0][0].shape[0]-1
    
    if ((n_comp != None) and (n_comp>max_comp)) or (n_comp==0):
        print(f"Number of components must be between 1 and {max_comp}")
        return
    
    preds, real = [], []
    var_explained = []
    
    for camp, df in Tab.items():
        data = pd.DataFrame(df[0].T)
        
        y = data.iloc[:, 0].values
        y_train = y[:-1].reshape(-1, 1)
        
        real.append(df[1])
        
        x = data.iloc[:, 1:].values
        x_train = x[:-1, :]
        x_test = x[-1, :].reshape(1, -1)
        
        sc = StandardScaler()
        pca = PCA()
        x_train_pca = pca.fit_transform(sc.fit_transform(x_train))
        
        mean_sum = P*scaling_factor*sc.mean_
        std_sum = np.sqrt(P*scaling_factor)*sc.scale_
        x_test = (x_test-mean_sum)/std_sum
        x_test_pca = pca.transform(x_test)  # test how x_test is transformed
        
        if n_comp!=None:
            trained = model.fit(x_train_pca[:, :n_comp].reshape(P, n_comp), y_train)
            pred = float(trained.predict(x_test_pca[:, :n_comp].reshape(1, n_comp))[0])
            preds.append(pred)
            var_explained.append(float(np.cumsum(pca.explained_variance_ratio_)[n_comp-1]))
        else:
            for comp in range(max_comp):
                trained = model.fit(x_train_pca[:, :comp+1].reshape(P, comp+1), y_train)
                pred = float(trained.predict(x_test_pca[:, :comp+1].reshape(1, comp+1))[0])
                if comp==0:
                    preds.append([pred])
                    var_explained.append([float(np.cumsum(pca.explained_variance_ratio_)[comp])])
                else:
                    preds[-1].append(pred) 
                    var_explained[-1].append(float(np.cumsum(pca.explained_variance_ratio_)[comp]))
                
    real_values = np.array(real)
    pred_values = np.array(preds).reshape(n_campaigns, ) if n_comp!=None else np.reshape(preds, (n_campaigns, max_comp))
    var_explained = np.array(var_explained) if n_comp!=None else np.reshape(var_explained, (n_campaigns, max_comp))
    
    if n_comp!=None:
        r2 = r2_score(real_values, pred_values)
        rmse = np.sqrt(mean_squared_error(real_values, pred_values))
        print(f"\n SNN based on PCR with {int(n_comp)} component(s)\n")
        print(f"Percentage of variance explained: {var_explained.mean(): .2%}")
        print(f"R2 score across {n_campaigns} campaigns: {r2: .5%}")
        print(f"Average RMSE score across {n_campaigns} campaigns: {rmse}")
    else:
        lst_r2 = [r2_score(real_values, pred_values[:, i]) for i in range(max_comp)]
        lst_rmse = [np.sqrt(mean_squared_error(real_values, pred_values[:, i])) for i in range(max_comp)]
        var_expl = var_explained.mean(axis=0)
        assert np.argmax(lst_r2)==np.argmin(lst_rmse), "check"
        
        print(f"\n SNN with analysis on number of components, {n_campaigns} campaigns \n")
        print(f" Highest R2 and RMSE score are obtained with {np.argmax(lst_r2)+1} component(s)")
        for i in range(1, max_comp+1):
            print(f"\n Principal component(s) selected: {int(i)} \n R2 score: {lst_r2[i-1]: .5%}")
            print(f" RMSE score: {lst_rmse[i-1]} \n Percentage of variance explained: {var_expl[i-1]: .2%}")
            
        fig, ax = plt.subplots(1, 1, figsize=(14, 5))
        plt.plot(list(range(1, max_comp+1)), lst_r2)
        plt.xlabel('Number of Principal Components')
        plt.ylabel(r'$R^2$ score')
        plt.title(r"Evolution of $R^2$ with n. of PC selected")       
        
        
# x_test_stand = np.divide(x_test-(P*sc.mean_), np.sqrt(P)*sc.scale_) 
# x_test (vector of sum of cookies) is not normalized before PCA since would become 0, it is assumded to be normalized by P (or not=
# pca_lst = [PCA(n_components=n_comp)] if n_comp!=None else [PCA(n_components=int(i)) for i in range(1, max_comp+1)]

### Google's algorithm

In [10]:
def ADM(X_train, Y_train, N, L, sigma, threshold, prop_panel=1, lasso=0, _print_=True, compute_r2=False):   # lasso is a lambda parameter (float)
    '''
    (Timed) implementation of Google's Adaptive Dirac Mixture algorithm
    T: regressors, R: dependent
    N are iterations, L additional centers, sigma variance
    Threshold is the threshold consider to discard alphas, if None is dynamic with K (works well)
    compute_r2 is list of [X_test, Y_test]
    '''
    
    # storing
    fun_eval = []
    metrics = []
    th_list = []
    K_list = [1]
    J = X_train.shape[1]
    
    # INIT
    K = 1
    alpha = [1]
    centers_ = [[1 for i in range(J)]]

    # N. OF ITERATIONS
    for it_ in range(N):         
        if _print_:
            print(f"\n \t \t ITER {it_+1}")
        mus = centers_.copy()
        # For all additional centers considered
        for m in range(L):         
            pis = alpha.copy()
            acc_pis = [np.sum(pis[:i]) for i in range(1, len(pis)+1)]
            # assert np.isclose(acc_pis[-1], 1, atol = 4*1e-2), f"cum is: {acc_pis[-1]}" 
        
            # Stochastically select Gaussian from which to generate center (according to their probability)
            r = np.random.uniform(0, 1) # sample uniform
            k = 0
            for i, thresh in enumerate(acc_pis):
                if r < thresh:
                    k = i
                    break
            selected_mu = mus[k]  # center considered
            selected_cov = sigma**2*np.eye(J)   # variance, same for all centers
            # center is generated and appended
            samples = np.random.multivariate_normal(selected_mu, selected_cov)  
            centers_ += [samples]   # mus is old X
        K = K+L
        
        # take time
        start = timer()
        
        # objective function to minimize to find weights alphas
        # it is the Sum of Squared deviation between real value and predictions
        # the sum of the absolute values of parameters with the lasso parameter is considered if passed
        def objective(alphas): 
            aux = 0.0
            for i in range(X_train.shape[0]):
                S_aux=0.0
                for k in range(K):
                    S_aux += alphas[k]*(1-np.exp(-np.dot(centers_[k], X_train[i])))  # SSE
                aux += ((Y_train[i]-S_aux)**2)/float(X_train.shape[0])
            aux += lasso*np.sum(np.abs(alphas))  # Lasso penalization
            return aux
        
        def constraint_alpha(alphas):
            return -1+np.sum(alphas)  # sum of weights must be equal to 1
        
        def constraint_alpha_lasso(alphas):
            return -1+np.sum(np.abs(alphas))  # sum of abs weights must be equal to 1
        
        # initial guess for alpha are re-drawn from a Dirichlet distribution every iteration
        alpha_initial_guess=stats.dirichlet.rvs([0.5 for i in range(K)])[0].reshape(-1, 1)
        
        bound = [(0,1) for i in range(K)]  # bounds for the parameters (are probabilities)
        bound_lasso = [(-1,1) for i in range(K)]
        cons = ({"type" : "eq", "fun" :  constraint_alpha})
        cons_lasso = ({"type" : "eq", "fun" :  constraint_alpha_lasso})
        
        # Minimization
        # cons_trust = LinearConstraint(A = np.ones((1,K)), lb=1, ub=1)
        # method = 'trust-constr' SLOWER
        if lasso!=0:
            sol = minimize(objective, alpha_initial_guess.flatten(), method='slsqp', bounds=bound_lasso, constraints=cons_lasso)
        else:
            sol = minimize(objective, alpha_initial_guess.flatten(), method='slsqp', bounds=bound, constraints=cons)
        alpha = sol.x  # Alphas' values minimizing
        
        loss = float(sol.fun) if lasso==0 else float(sol.fun) - lasso*np.sum(np.abs(alpha))
        lasso_pen = 0 if lasso==0 else float(sol.fun - loss)
        fun_eval.append(loss)  # Storing evaluation of objective function
        
        if _print_:
            timer(start, 'minimization')  # print timing with description of task
            print(f"\nLOSS evaluation (without lasso penalization): {fun_eval[-1]}")
            
        # THRESHOLD, if not set, use Adaptive (proportion of exp value of probability
        sogl = 0.05*(1/K) if threshold==None else threshold  # if thresh None proportion of 1/K, if passed it is taken as such
        th_list.append(sogl)
        
        discard = 0
        new_alpha=[]
        new_centers=[]
        for q in range(len(alpha)):
            if alpha[q] < float(sogl): 
                discard += 1
                continue
            else:
                new_alpha += [alpha[q]]
                new_centers += [centers_[q]]
        
        # update after discarding and store
        K = len(new_alpha)
        K_list.append(K)
        alpha = new_alpha.copy()
        centers_ = new_centers.copy()
        
        if _print_:
            print(f"Considered centers: {len(centers_)}")
            print(f"Discarded centers: {discard}")  # print how many discarded centers
            
        if type(compute_r2)==list:
            preds, r2, rmse = eval_ADM(K_list, alpha, centers_, compute_r2[0], compute_r2[1], print_=_print_)
            metrics.append([r2, rmse])
            
        # stopping condition
        if it_== 0:
            best_alpha = alpha.copy()
            best_centers = centers_.copy()
            best_iter, disc = 0, 0
        else:
            if fun_eval[-1] < fun_eval[best_iter]:
                best_iter = it_
                best_alpha = alpha.copy()
                best_centers = centers_.copy()
                disc = 0
            else:
                disc += 1
                
        if (it_ > 34) and (disc > 9):
            alpha = best_alpha.copy()
            centers_ = best_centers.copy()
            K_list = K_list[:best_iter+2]  # since K_List has one value more (appended at the beginning)
            fun_eval = fun_eval[:best_iter+1]
            th_list = th_list[:best_iter+1]
            if type(compute_r2)==list:
                metrics = metrics[:best_iter+1]
            if _print_:
                print(f'\n\nSTOPPING condition at iter {it_+1}, considering best iter {best_iter+1}\n\n')
            break

    result = [alpha, centers_, K_list, fun_eval, th_list]
    if type(compute_r2)==list:
        result.insert(4, metrics)
    return result

# covs = [sigma**2*np.eye(J) for i in range(len(X))]

In [11]:
def comp_param(real_K, real_alph, pred_K, pred_alph):
    print(f"\nThe real n. of types is: {real_K}", f"\nThe predicted one is: {pred_K[-1]}\n")
    real_alph = [v for k, v in real_alph.items()]
    real_alph = list(np.reshape(real_alph, (-1, real_K)).mean(axis=0))
    pred_alph_vis = np.array(pred_alph)[np.argsort(pred_alph)][::-1].tolist()
    
    print(f"\nProportion of real user types (avg. across tableaux): \n", real_alph)
    print(f"Proportion of predicted user types (sorted): \n", pred_alph_vis, "\n")
    
def eval_ADM(pred_K, pred_alph, centers, X_test, Y_test, plot_=False, print_=True, real_K=False, real_alph=False):
    '''
    Function to evaluate the Adaptive Dirac Mixture Algorithm, computing r2 and rmse
    pred_K: list of all K predicted
    pred_alph: list of all weights predicted
    plot_: string (name of file to save 3d-plotted centers to)
    '''
    if (real_K==True) and (real_alph==True):
        comp_param(real_K, real_alph, pred_K, pred_alph)
    
    preds = []
    for i in range(X_test.shape[0]):
        pt = 0.0
        for k in range(pred_K[-1]):
            pt += pred_alph[k]*np.exp(-np.dot(centers[k], X_test[i]))
        preds.append(1-pt)  # storing predictions
    preds = np.array(preds)
    r2 = r2_score(Y_test, preds)
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
    if print_:
        print(f"\nThe R2 score is: {r2: .5%}", "-"*5, f"The RMSE is: {rmse}", "-"*5, "\n")
    if type(plot_)==str:
        plot_centers(centers, plot_)
    return preds, r2, rmse

def plot_centers(centers_, name_file):
    J = len(centers_[0])
    centers_ = np.reshape(centers_, (len(centers_), J))
    proj_centers = PCA(n_components=3).fit_transform(centers_)
    
    fig = plt.figure(figsize = (10, 10))
    ax = plt.axes(projection ="3d")
 
    # Creating plot
    ax.scatter3D(proj_centers[:, 0], proj_centers[:, 1], proj_centers[:, 2], s=70, c = np.abs(proj_centers[:, 2]), cmap="RdYlGn")   #  color = "green"
                          
    ax.set_title("3D Scatter plot of fitted centers", fontsize=18)
    ax.set_xlabel('PCA-One', fontsize=12)
    ax.set_ylabel('PCA-Two', fontsize=12)
    ax.set_zlabel('PCA-Three', fontsize=12)
    
    fig.savefig('images/' + name_file + '.jpg', bbox_inches='tight', dpi=300)   
                          
    plt.show()

In [12]:
def plot_ADM(K_list, func_eval, thresh, lasso, r2_ls = False, th_list=False): 
    '''
    Function to plot evolution of number of centers considered (K) and associated Loss evaluations, for each iteration considered
    K_list: list of all numbers of centers considered between iteration
    func_eval: list of Loss evaluations, for each iteration (excluding lasso penalization)
    th_list: list of threshold used
    r2_ls: list of [r2, rmse] to be plotted
    thresh and lasso: only for visualization
    '''
    
    K_list = K_list[1:]  # not considering Init value (1)
    assert len(K_list) == len(func_eval), 'to check' # indeed they differ (without above) since K_list is inizialied with one value at Init (it is longer by 1)
    iterations = np.array(list(range(1, len(func_eval)+1)))
    if type(r2_ls)==list:
        r2_ls = np.reshape(r2_ls, (len(K_list), 2))
    
    var = [K_list, r2_ls] if type(r2_ls)==np.ndarray else [K_list, func_eval]
    tit = ['predicted K',  r'$R^2$ and RMSE'] if type(r2_ls)==np.ndarray else ['predicted K', 'Loss evaluations']
    lab = ['K values', r'$R^2$ and RMSE'] if type(r2_ls)==np.ndarray else ['K values', 'Mean Squared Error']
    
    if th_list:
        var.append(th_list)
        tit.append('thresholds used')
        lab.append('Threshold')
    
    if th_list:
        fix, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 8*3))
        axs = [ax1, ax2, ax3]
    else:
        fix, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8*2))
        axs = [ax1, ax2]
    to_plot = [var, tit, lab]
    # plotting and visualizatiom for titles
    for idx, ax in enumerate(axs):            
        if idx==0:
            ax.plot(iterations, to_plot[0][idx], color="blue")
            ax.set_ylabel(f"{to_plot[2][idx]}", fontsize=14, color="blue")
        elif idx==1 and type(r2_ls)!=np.ndarray:
            ax.plot(iterations, to_plot[0][idx], color="red")
            ax.set_ylabel(f"{to_plot[2][idx]}", fontsize=14, color="red")
            
        elif idx==1 and type(r2_ls)==np.ndarray:
            ax2 = ax.twinx()
            
            ax.plot(iterations, to_plot[0][idx][:, 0], color="darkcyan", label=f"$R^2$")  # r2
            ax.set_ylim(bottom=-1, top=np.amax(to_plot[0][idx][:, 0]))
            ax.set_yticks(ax.get_yticks().tolist())
            ax.set_yticklabels([f'{x: .1%}' for x in  ax.get_yticks().tolist()])
            ax.legend(loc='upper right', fontsize='large')
            ax.set_ylabel(f"$R^2$", fontsize=15, color="darkcyan")
            
                    
            ax2.plot(iterations, to_plot[0][idx][:, 1], color="crimson", label="RMSE")  # rmse
            ax2.legend(loc='lower right', fontsize='large')
            ax2.set_ylabel("RMSE", fontsize=14, color="crimson")
        else:
            ax.plot(iterations, to_plot[0][idx], color="orangered")
            ax.set_ylabel(f"{to_plot[2][idx]}", fontsize=14, color="orangered")
        
        title_for_thresh = "with adaptive thresh" if thresh==None else f"thresh = {thresh}"  #{thresh: .4f}
        title_for_lasso = "without lasso pen" if lasso==0 else f"lasso pen={lasso}"   #{lasso: .4f}
        
        if idx==0:
            ax.set_title(f"Evolution of {to_plot[1][idx]}, {title_for_lasso} and {title_for_thresh}", fontsize=15)
        else:
            ax.set_title(f"Evolution of {to_plot[1][idx]}", fontsize=15)
        ax.set_xlabel("Number of iterations", fontsize=14)
        
    plt.show()
    return fix

### Initial study of simulated dataset

In [13]:

def time_creation(P, J, K, MoyLCM, Mass0, N_tableaux):
    '''
    Function to display the time for creation of dataset
    '''
    start = timer()                # st = time.process_time()
    T, Types, theta, alpha_proportion, poids_0 = dataset_LCM(P, J, K, MoyLCM, Mass0, N_tableaux)  # random poids in [0, 1]
    timer(start, "creating the Tableaux")        # elapsed = time.process_time() - st, print('CPU time:', elapsed, 'seconds')
    return T, Types, theta, alpha_proportion, poids_0    

def export_gen(P, J, K, MoyLCM, Mass0, N_tableaux, to_store=5):
    '''
    Fuction to store as an excel file a sample of 5 (default) dataset
    it comprises the Theta matrix (parameters of Poisson), in common among all subcampaigns as well as Alpha proportions (varying between sub-campaigns)
    '''
    FilePath = os.path.realpath("Visualize.xlsx")
    #ExcelWorkbook = openpyxl.load_workbook(FilePath)
    writer = pd.ExcelWriter(FilePath, engine = 'openpyxl')
    T, Types, theta, alpha_proportion, poids_0 = dataset_LCM(P, J, K, MoyLCM, Mass0, N_tableaux)  # random poids in [0, 1]
    pad_vert = pd.DataFrame(np.zeros((P, 1)), columns=[""])
    pad_vert.replace(0, np.nan, inplace=True)
    pad_1 = np.zeros((P-K, J))
    pad_1[:] = np.nan
    # just stacking arrays for visualization
    left = pd.DataFrame(data=np.vstack((theta, pad_1)), columns=["theta cookie " + str(x + 1) for x in range(J)])
    
    pad_2 = pd.DataFrame(np.zeros((P-1, K)), index=np.arange(1, P))
    pad_2[:] = np.nan
    
    for i in range(to_store):
        alpha = alpha_proportion[i].reshape(1, -1)
        alpha = pd.DataFrame(data = alpha, columns=['proportion ' + str(x) for x in range(K)])
        right = pd.concat([alpha, pad_2], axis=0, ignore_index=True)
        right = right.iloc[:, :K]
        result = pd.concat([T[i], pad_vert, left, pad_vert, right], axis=1, ignore_index=True)
        result.columns = T[i].columns.to_list() + pad_vert.columns.to_list() + left.columns.to_list() + pad_vert.columns.to_list() + alpha.columns.to_list()
        result.to_excel(writer, sheet_name='Sheet '+str(i+1), na_rep='', index=False, columns=result.columns.to_list())
    writer.save()
    writer.close()
    return T, Types, theta, alpha_proportion, poids_0

def export_data(X, Y):
    '''
    Function to store in an excel file the data fed to the models (both regressor and dependent)
    '''
    n_tableau = X.shape[0]
    assert X.shape[0]==Y.shape[0], 'incorrect data format'
    FilePath = os.path.realpath("Dataset.xlsx")
    writer = pd.ExcelWriter(FilePath, engine = 'openpyxl')
    
    pad_vert = pd.DataFrame(np.zeros((n_tableau, 1)), columns=[""])
    pad_vert.replace(0, np.nan, inplace=True)
    result = pd.concat([X, pad_vert, Y], axis=1, ignore_index=True)
    result.columns = X.columns.to_list() + pad_vert.columns.to_list() + Y.columns.to_list()
    result.to_excel(writer, sheet_name='Data', na_rep='', index=False, columns=result.columns.to_list())
        
    writer.save()
    writer.close()

### Additional Code

In [14]:
# K, P = 3, 20
# True_Z=np.random.randint(0,K,size=P)
# alpha_proportion = {}
# alpha_proportion=[0 for i in range(K)]
# Z=True_Z
# print(Counter(Z))
# for i in Counter(Z):   # sorted
#    print(i)
#    alpha_proportion[i]=Counter(Z)[i]/len(Z)   

In [15]:
# T_aux, cs, reach, lsum = result[0]
# cs = np.array(cs, dtype=int).reshape(1, J)
# reach = np.array(reach, dtype=float).reshape(1, 1)

# start = timer()
# for i in range(1, len(result)):
#     t, c, rec, lsum = result[i]
#     cs = np.vstack([cs, np.array(c, dtype=int).reshape(1, J)])
#     reach = np.vstack([reach, np.array(rec, dtype=float).reshape(1, 1)])

# X = pd.DataFrame(data=cs, columns=['cookie ' + str(i+1) for i in range(J)])
# Y = pd.DataFrame(data=reach, columns=['reach estimate'])
# timer(start)
# print(X.shape)
# print(Y.shape)

In [16]:
# NOTES FOR THRESHOLD

# for threshold

#decimal=1
#while P/10**decimal>=10:
#    decimal+=1
#for k in range(K):
#        if alpha[k]<10**(-decimal):   # threshold
#            alpha[k]=0

#decimal=1
#while K/decimal>=1:
    #decimal+=1
#print(f"threshold: {K*10**(-decimal): .2f}")
#for q in range(len(alpha)):
#    if alpha[q]<K*10**(-decimal):   # threshold
#        alpha[q]==0


#decimal = 0
#while K*10/10**decimal>=1:
    #decimal+=1
#threshold = 10**(-decimal)*(K-1)
#print(f"threshold: {threshold}")

#threshold = float(1/K)

#with threshold = float(1/K**2) R2 -0.000008 and RMSE 0.02

### print variance of explained by components selected

def snn(Tab, model, n_comp=None):
    '''
    function implementing Synthethic Nearest Neighbour algorithm for our specific setup
    Tab: dict result of preprocessing (with keys=campaigns)
    model: sklearn estimator, with fit and predict methods
    n_components: if None all components are tried and the evolution of mean squared error is plotted
                  if fixed to a number (say n), considered only first n components
    '''
    r2, rmse = [], []
    
    P = Tab[0][0].shape[1]-1
    n_campaigns = len(Tab)
    max_comp = Tab[0][0].shape[0]-1
    
    if ((n_comp != None) and n_comp>max_comp) or ((n_comp != None) and n_comp<1):
        print(f"The number of components must be between 1 and {max_comp}")
        return
    
    pca_lst = [PCA(n_components=n_comp)] if n_comp!=None else [PCA(n_components=int(i)) for i in range(1, max_comp+1)]
    preds, real = [], []
    
    for camp, df in Tab.items():
        data = pd.DataFrame(df[0].T)
        
        y = data.iloc[:, 0].values
        y_train = y[:-1].reshape(-1, 1)
        
        real.append(df[1])
        
        x = data.iloc[:, 1:].values
        x_train = x[:-1, :]
        x_test = x[-1, :].reshape(1, -1)
        
        for pca in pca_lst:            
            sc = StandardScaler()
            x_train_pca = pca.fit_transform(sc.fit_transform(x_train))
            x_test_pca = pca.transform(x_test)  # test how x_test is transformed
            trained = model.fit(x_train_pca, y_train)
            pred = float(trained.predict(x_test_pca)[0])
            if len(pca_lst)>1 and pca==pca_lst[0]:
                preds.append([pred])
            elif len(pca_lst)>1:
                preds[-1].append(pred)
            else:
                preds.append(pred)
                
    real_values = np.array(real)
    pred_values = np.array(preds).reshape(n_campaigns, 1) if n_comp!=None else np.reshape(preds, (n_campaigns, len(pca_lst)))
    
    if n_comp!=None:
        r2 = r2_score(real_values, pred_values)
        rmse = np.sqrt(mean_squared_error(real_values, pred_values))
        print(f"\n SNN based on PCR with {int(n_comp)} component(s)\n")
        print(f"R2 score across {n_campaigns} campaigns: {r2: .5%}")
        print(f"Average RMSE score across {n_campaigns} campaigns: {rmse}")
    else:
        lst_r2 = [r2_score(real_values, pred_values[:, i]) for i in range(len(pca_lst))]
        lst_rmse = [np.sqrt(mean_squared_error(real_values, pred_values[:, i])) for i in range(len(pca_lst))]
        assert np.argmax(lst_r2)==np.argmin(lst_rmse), "check"
        
        print(f"\n SNN with analysis on number of components, {n_campaigns} campaigns")
        print(f"Highest R2 and RMSE score are obtained with {np.argmax(lst_r2)+1} component(s)")
        for i in range(1, max_comp+1):
            print(f"\n{int(i)} Principal component(s) selected \n R2: {lst_r2[i-1]: .5%} \n RMSE: {lst_rmse[i-1]}\n")
            
        fig, ax = plt.subplots(1, 1, figsize=(14, 5))
        plt.plot(list(range(1, max_comp+1)), lst_r2)
        plt.xlabel('Number of Principal Components')
        plt.ylabel(r'$R^2$ score')
        plt.title(r"Evolution of $R^2$ with n. of PC selected")       
        
        
#x_test_stand = np.divide(x_test-(P*sc.mean_), np.sqrt(P)*sc.scale_) 
#x_test (vector of sum of cookies) is not normalized before PCA since would become 0, it is assumded to be normalized by P (or not=
    

### function for snn data engineering select all P

def snn_dataset_sim(Tableau, normalize_sum_reach=False):
    '''
    (Timed) function that takes as input the simulated dataset (dict with keys=campaign ID)
    used to add 1st row filled with {0, 1} and last column with sum of collect cookies (for each j in J since transposed)
    
    the first value in the last column (position 0) is therefore the reach computed as int, which is stored and 
    substituted with a NaN value to be predicted by the SNN algorithm
    
    normalize_sum_reach: bool, whether to normalized last column (c mapped to t, and reach to reach/P)
    '''
    
    P = Tableau[0].shape[0]
    J = Tableau[0].shape[1]-1  # there is 'user' column
    
    # start timer
    start = timer()
    result = {}
    
    # computations reach and vector of counts for each sub-campaign
    for n in range(len(Tableau)):
        T_aux = Tableau[n].to_numpy().T  # transposed
        T_aux = discretize_columns(T_aux)  # discretization (first row)
        
        vector_sum=np.nansum(T_aux, axis=1).reshape((-1, 1))
        if normalize_sum_reach:
            vector_sum = np.divide(vector_sum, T_aux.shape[1])   # normalized t_i vector and reach 
        
        reach_comp = float(vector_sum[0, 0])
        vector_sum[0, 0] = np.nan        
        final = np.hstack((T_aux, vector_sum))
        
        # result stores everything, separately we build regressors (normalized vector of cookies count) and dependent (calculation of reach function)
        result[n]=(final, reach_comp)
    
    # visualization of timer with description of task
    timer(start, "preprocessing the dataset for SNN")    
    return result


def snn_dataset_sim(Tableau, normalize_sum_reach=False, P_panelist=100):
    '''
    (Timed) function that takes as input the simulated dataset (dict with keys=campaign ID)
    used to add 1st row filled with {0, 1} and last column with sum of collect cookies (for each j in J since transposed)
    
    the first value in the last column (position 0) is therefore the reach computed as int, which is stored and 
    substituted with a NaN value to be predicted by the SNN algorithm
    
    normalize_sum_reach: bool, whether to normalized last column (c mapped to t, and reach to reach/P)
    '''
    
    P = Tableau[0].shape[0]
    P_panelist = P if P<P_panelist else P_panelist
    J = Tableau[0].shape[1]-1  # there is 'user' column
    
    # start timer
    start = timer()
    result = {}
    
    # computations reach and vector of counts for each sub-campaign
    for n in range(len(Tableau)):
        T_aux = Tableau[n].to_numpy().T  # transposed
        T_aux = discretize_columns(T_aux[1:, :])  # discretization (first row)
        
        vector_sum=np.nansum(T_aux, axis=1).reshape((-1, 1))
        if normalize_sum_reach:
            vector_sum = np.divide(vector_sum, P)   # normalized t_i vector and reach 
        
        reach_comp = float(vector_sum[0, 0])
        vector_sum[0, 0] = np.nan   
        
        slice_columns = np.arange(P, dtype=int)   # not taken care of rappresentation of group K
        random_cols = np.random.choice(slice_columns, size=P_panelist, replace=False)
        T_panelist = T_aux[:, random_cols]
           
        final = np.hstack((T_panelist, vector_sum))
        
        # result stores everything, separately we build regressors (normalized vector of cookies count) and dependent (calculation of reach function)
        result[n]=(final, reach_comp)
    
    # visualization of timer with description of task
    timer(start, "preprocessing the dataset for SNN")    
    return result

In [17]:
# stopping condition

#check = pd.Series(data=fun_eval, dtype=float).diff()
#print(f"Considered: {np.abs(check.tail(5).array).reshape(-1, 1).tolist()}")
#if np.any(np.abs(check.tail(5).array).reshape(-1, 1) < 2e-5, axis=1).sum() > 3:   # norm of difference between Loss evaluations of last 3 iterations must be all less than 0.00002
    #or np.any(np.abs(check.tail(3).array).reshape(-1, 1) < 2e-3, axis=1).sum() > 2
    #if print_:
        #print(f'\n\nSTOPPING condition (stability of loss) at iter {b}\n\n')
    #break
            
#if len(fun_eval) > 4:
    #if (fun_eval[-4] < 0.1*float(round(fun_eval[-5], 2))) and (float(round(fun_eval[-5], 2)<1)):
        #if print_:
            #print(f'\n\nSTOPPING condition (big initial drop) at iter {b}\n\n')
        #break